# Generate pm4py log using ASP Generator

ASP log generator uses the decl model which converts the model into abudction logic programming and pass to the clingo.
Clingo generates the output which is turned into the pm4py log or can be created a .xes file.


In [1]:
!python --version

Python 3.9.12


In [23]:
import sys
import os
import logging
import pathlib

SCRIPT_DIR = pathlib.Path("../../../", "src").resolve()
sys.path.append(os.path.dirname(SCRIPT_DIR))

from src.Declare4Py.ProcessMiningTasks.ASPLogGeneration.asp_generator import AspGenerator
from src.Declare4Py.ProcessModels.DeclareModel import DeclareModel

In [24]:
# Setting up required files

# possible files to test
decl_model_1 = "./decl_files/Response.decl"
decl_model_2 = "./decl_files/MikeModel.decl"
decl_model_3 = "./decl_files/reference10.decl"
decl_model_4 = "./decl_files/diagonisis.decl"

decl_file = os.path.join(pathlib.Path(decl_model_2).resolve())
output_file = os.path.join(pathlib.Path("./generated_exporter.xes").resolve())

In [25]:
# Create Declare model by reading declare model from a file.

model: DeclareModel = DeclareModel().parse_from_file(decl_file)

In [31]:
#general Setting

# Number of traces that should be generated
num_of_traces = 4

# Minimum and maximum number of events a trace can contain
(num_min_events, num_max_events) = (2, 14)


In [32]:
# Initializing ASP generator with default distributor which is uniform.

logging.basicConfig(level=logging.DEBUG)
asp = AspGenerator(
    model,
    num_of_traces,
    num_min_events,
    num_max_events,
)


DEBUG:ASP generator:Distribution for traces uniform
DEBUG:ASP generator:traces: 4, events can have a trace min(2) max(14)
INFO:ASP generator:Computing distribution
DEBUG:Distributor:Distribution() uniform min_mu: 2 max_sigma: 14 num_traces: 4 custom_prob: None
DEBUG:Distributor:Uniform() probabilities: [Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13)]
DEBUG:Distributor:Custom_dist() min_mu:2 max_sigma:14 num_traces:4
DEBUG:Distributor:Probabilities sum 1
DEBUG:Distributor:Distribution result: [11  5 11 11]
INFO:ASP generator:Distribution result Counter({11: 3, 5: 1})


In [33]:
# activity A
# bind A: attribute
# activity B
# activity C
# attribute: value1, value2, value3
# Response[A,B] | A.attribute is value1 | |


# A user may want the activation of the Response constraint (i.e., activity A with
# attribute attribute = value1) to happen at least 3 times within each generated trace.


asp.set_activation_conditions({
    'Response[Driving_Test, Resit] |A.Grade<=2 | |': [0, 3]  # activation should occur between at most 3 times
})

asp.set_activation_conditions({'Response[Driving_Test, Resit] |A.Grade<=2 | |': [3, 5]})  # activation should occur between 3 to 5 times

asp.set_activation_conditions_by_template_index({1: [2, 4]})  # activation should occur between 3 to 5 times and as key you can also define the template index instead str


In [34]:
# Generate the traces and parse the result produced by clingo
asp.run()  # Run accets 1 optional value whether to create file for the ASP generated from given declare model


DEBUG:ASP generator:Using custom traces length
INFO:ASP generator:Computing distribution
DEBUG:Distributor:Distribution() uniform min_mu: 2 max_sigma: 14 num_traces: 4 custom_prob: None
DEBUG:Distributor:Uniform() probabilities: [Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13)]
DEBUG:Distributor:Custom_dist() min_mu:2 max_sigma:14 num_traces:4
DEBUG:Distributor:Probabilities sum 1
DEBUG:Distributor:Distribution result: [4 2 2 8]
INFO:ASP generator:Distribution result Counter({2: 2, 4: 1, 8: 1})
INFO:ASP generator:Computing distribution
DEBUG:Distributor:Distribution() uniform min_mu: 2 max_sigma: 14 num_traces: 0 custom_prob: None
DEBUG:Distributor:Uniform() probabilities: [Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(1, 13), Fraction(

In [35]:
# Save file to xes
asp.to_xes(output_file)

exporting log, completed traces :: : 0it [00:00, ?it/s]